# Task 1: Exploratory Data Analysis (EDA) - Financial News Sentiment

**Objective**: Discover patterns in financial news data and understand relationships between news sentiment and stock movements.

**Key Focus Areas**:
1. Dataset structure and quality assessment
2. Descriptive statistics on textual content
3. Publisher distribution and activity patterns
4. Temporal trends in news publication
5. Topic modeling and keyword extraction
6. Statistical distributions and insights

**Timeline**: Week 1 (Nov 19-25, 2025)
**Challenge**: Nova Financial Solutions - Predicting Price Moves with News Sentiment

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# For text processing
from collections import Counter
from datetime import datetime, timedelta
import re

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Create sample financial news dataset for demonstration
# In production, load your actual data: df = pd.read_csv('../data/raw/news_data.csv')

np.random.seed(42)

# Sample stocks and publishers
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM']
publishers = ['Reuters', 'Bloomberg', 'CNBC', 'MarketWatch', 'Seeking Alpha', 
              'Financial Times', 'Yahoo Finance', 'The Wall Street Journal']

# Generate sample data
n_records = 500
dates = pd.date_range(start='2025-10-01', periods=n_records, freq='D')
np.random.shuffle(dates)

sample_headlines = [
    "Apple stock hits new high amid strong iPhone sales",
    "Microsoft reports record quarterly earnings",
    "Tech stocks rally on AI optimism",
    "FDA approves new drug, stock soars",
    "Market volatility increases amid rate concerns",
    "Amazon raises prices on Prime membership",
    "Tesla misses earnings expectations",
    "Intel cuts production guidance",
    "Google search updates impact market",
    "Stock market shows resilience after dip"
]

data = {
    'headline': np.random.choice(sample_headlines, n_records),
    'url': [f"https://example.com/article-{i}" for i in range(n_records)],
    'publisher': np.random.choice(publishers, n_records),
    'date': dates,
    'stock': np.random.choice(stocks, n_records)
}

df_news = pd.DataFrame(data)
df_news['date'] = pd.to_datetime(df_news['date'])
df_news = df_news.sort_values('date').reset_index(drop=True)

print(f"✓ Loaded {len(df_news)} news articles")
print(f"Date range: {df_news['date'].min()} to {df_news['date'].max()}")
print(f"\nDataset shape: {df_news.shape}")
print(f"\nFirst few rows:")
df_news.head()

In [ ]:
# Check data types
print("=" * 60)
print("DATA TYPES")
print("=" * 60)
print(df_news.dtypes)

# Check for missing values
print("\n" + "=" * 60)
print("MISSING VALUES")
print("=" * 60)
missing_data = pd.DataFrame({
    'Column': df_news.columns,
    'Missing_Count': df_news.isnull().sum(),
    'Missing_Percentage': (df_news.isnull().sum() / len(df_news) * 100).round(2)
})
print(missing_data)

# Check for duplicates
print("\n" + "=" * 60)
print("DUPLICATE RECORDS")
print("=" * 60)
duplicate_count = df_news.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")

# Data size
print("\n" + "=" * 60)
print("DATASET SIZE")
print("=" * 60)
print(f"Total records: {len(df_news):,}")
print(f"Total columns: {len(df_news.columns)}")
print(f"Memory usage: {df_news.memory_usage(deep=True).sum() / 1024:.2f} KB")

In [ ]:
# Text feature extraction
df_news['headline_length'] = df_news['headline'].str.len()
df_news['word_count'] = df_news['headline'].str.split().str.len()
df_news['unique_words'] = df_news['headline'].str.lower().str.split().apply(lambda x: len(set(x)))

print("=" * 60)
print("HEADLINE STATISTICS")
print("=" * 60)
text_stats = pd.DataFrame({
    'Metric': ['Count', 'Mean', 'Std Dev', 'Min', 'Max', 'Median'],
    'Headline Length': [
        df_news['headline_length'].count(),
        df_news['headline_length'].mean(),
        df_news['headline_length'].std(),
        df_news['headline_length'].min(),
        df_news['headline_length'].max(),
        df_news['headline_length'].median()
    ],
    'Word Count': [
        df_news['word_count'].count(),
        df_news['word_count'].mean(),
        df_news['word_count'].std(),
        df_news['word_count'].min(),
        df_news['word_count'].max(),
        df_news['word_count'].median()
    ]
})
print(text_stats.to_string(index=False))

# Visualize headline length distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(df_news['headline_length'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Headline Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Headline Lengths')
axes[0].axvline(df_news['headline_length'].mean(), color='red', linestyle='--', label=f"Mean: {df_news['headline_length'].mean():.0f}")
axes[0].legend()

axes[1].hist(df_news['word_count'], bins=20, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Words per Headline')
axes[1].axvline(df_news['word_count'].mean(), color='darkred', linestyle='--', label=f"Mean: {df_news['word_count'].mean():.1f}")
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n✓ Text analysis complete")

In [ ]:
# Publisher statistics
publisher_counts = df_news['publisher'].value_counts()
print("=" * 60)
print("PUBLISHER ANALYSIS")
print("=" * 60)
print(f"Total unique publishers: {df_news['publisher'].nunique()}")
print("\nArticles per publisher:")
print(publisher_counts)

# Publisher statistics table
print("\n" + "=" * 60)
print("PUBLISHER STATISTICS")
print("=" * 60)
publisher_stats = df_news.groupby('publisher').agg({
    'headline': 'count',
    'stock': 'nunique',
    'headline_length': 'mean'
}).round(2)
publisher_stats.columns = ['Total_Articles', 'Unique_Stocks', 'Avg_Headline_Length']
publisher_stats = publisher_stats.sort_values('Total_Articles', ascending=False)
print(publisher_stats)

# Visualize publisher distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot
publisher_counts.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Number of Articles')
axes[0].set_ylabel('Publisher')
axes[0].set_title('Articles per Publisher')
axes[0].invert_yaxis()

# Pie chart
colors = plt.cm.Set3(range(len(publisher_counts)))
axes[1].pie(publisher_counts.values, labels=publisher_counts.index, autopct='%1.1f%%', colors=colors)
axes[1].set_title('Publisher Contribution Distribution')

plt.tight_layout()
plt.show()

print("\n✓ Publisher analysis complete")

In [ ]:
# Stock coverage analysis
stock_counts = df_news['stock'].value_counts()
print("=" * 60)
print("STOCK COVERAGE ANALYSIS")
print("=" * 60)
print(f"Total unique stocks: {df_news['stock'].nunique()}")
print("\nArticles per stock:")
print(stock_counts)

# Stock statistics
stock_stats = df_news.groupby('stock').agg({
    'headline': 'count',
    'publisher': 'nunique',
    'date': ['min', 'max'],
    'headline_length': 'mean'
}).round(2)
stock_stats.columns = ['Article_Count', 'Unique_Publishers', 'First_Date', 'Last_Date', 'Avg_Headline_Length']
stock_stats = stock_stats.sort_values('Article_Count', ascending=False)
print("\n" + "=" * 60)
print("STOCK STATISTICS")
print("=" * 60)
print(stock_stats)

# Visualize stock coverage
fig = plt.figure(figsize=(14, 6))
stock_counts.plot(kind='bar', color='darkgreen', alpha=0.7, edgecolor='black')
plt.xlabel('Stock Ticker')
plt.ylabel('Number of Articles')
plt.title('News Coverage by Stock Symbol')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n✓ Stock coverage analysis complete")

In [ ]:
# Temporal analysis
df_news['date_only'] = df_news['date'].dt.date
daily_counts = df_news.groupby('date_only').size()

print("=" * 60)
print("TEMPORAL ANALYSIS")
print("=" * 60)
print(f"Date range: {df_news['date'].min()} to {df_news['date'].max()}")
print(f"Total days with news: {len(daily_counts)}")
print(f"Average articles per day: {daily_counts.mean():.2f}")
print(f"Max articles in a day: {daily_counts.max()}")
print(f"Min articles in a day: {daily_counts.min()}")

# Weekly analysis
df_news['year_week'] = df_news['date'].dt.isocalendar().year.astype(str) + '-W' + df_news['date'].dt.isocalendar().week.astype(str).str.zfill(2)
weekly_counts = df_news.groupby('year_week').size()

print(f"\nArticles per week (avg): {weekly_counts.mean():.2f}")

# Visualize temporal trends
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Daily trend
axes[0].plot(daily_counts.index, daily_counts.values, marker='o', linestyle='-', color='steelblue', alpha=0.7)
axes[0].fill_between(range(len(daily_counts)), daily_counts.values, alpha=0.3, color='steelblue')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of Articles')
axes[0].set_title('Daily News Publication Frequency')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Weekly trend
axes[1].bar(range(len(weekly_counts)), weekly_counts.values, color='coral', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Week')
axes[1].set_ylabel('Number of Articles')
axes[1].set_title('Weekly News Publication Frequency')
axes[1].set_xticks(range(0, len(weekly_counts), max(1, len(weekly_counts)//10)))
axes[1].set_xticklabels(weekly_counts.index[::max(1, len(weekly_counts)//10)], rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ Temporal analysis complete")

In [ ]:
# Extract keywords from headlines
def extract_keywords(text, min_length=3):
    """Extract meaningful words from text"""
    # Remove punctuation and convert to lowercase
    words = re.sub(r'[^\w\s]', '', text.lower()).split()
    # Filter by length and remove common stopwords
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'is', 'are'}
    return [w for w in words if len(w) >= min_length and w not in stopwords]

# Get all keywords
all_keywords = []
for headline in df_news['headline']:
    all_keywords.extend(extract_keywords(headline))

keyword_counts = Counter(all_keywords)
top_keywords = keyword_counts.most_common(20)

print("=" * 60)
print("TOP KEYWORDS IN HEADLINES")
print("=" * 60)
for keyword, count in top_keywords:
    print(f"{keyword:20} : {count:4} occurrences")

# Visualize top keywords
keywords, counts = zip(*top_keywords)
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(keywords, counts, color='purple', alpha=0.7, edgecolor='black')
ax.set_xlabel('Frequency')
ax.set_ylabel('Keyword')
ax.set_title('Top 20 Keywords in Financial News Headlines')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\n✓ Keyword analysis complete")

In [ ]:
# Create a cross-tabulation analysis
print("=" * 60)
print("PUBLISHER-STOCK COVERAGE MATRIX (Top Publishers & Stocks)")
print("=" * 60)

top_pubs = df_news['publisher'].value_counts().head(5).index
top_stocks = df_news['stock'].value_counts().head(5).index

crosstab = pd.crosstab(df_news[df_news['publisher'].isin(top_pubs)]['publisher'],
                        df_news[df_news['stock'].isin(top_stocks)]['stock'])
print(crosstab)

# Heatmap visualization
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(crosstab, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Article Count'}, ax=ax)
ax.set_title('Publisher-Stock Coverage Matrix')
ax.set_xlabel('Stock Symbol')
ax.set_ylabel('Publisher')
plt.tight_layout()
plt.show()

# Average headline length by stock
print("\n" + "=" * 60)
print("AVERAGE HEADLINE LENGTH BY STOCK")
print("=" * 60)
avg_length_by_stock = df_news.groupby('stock')['headline_length'].mean().sort_values(ascending=False)
print(avg_length_by_stock.round(2))

print("\n✓ Cross-sectional analysis complete")

In [ ]:
print("=" * 70)
print("EXECUTIVE SUMMARY - EXPLORATORY DATA ANALYSIS")
print("=" * 70)

summary = f"""
1. DATASET OVERVIEW
   • Total articles: {len(df_news):,}
   • Date range: {df_news['date'].min().date()} to {df_news['date'].max().date()}
   • Unique stocks covered: {df_news['stock'].nunique()}
   • Unique publishers: {df_news['publisher'].nunique()}
   • Data completeness: {(1 - df_news.isnull().sum().sum() / (len(df_news) * len(df_news.columns))) * 100:.1f}%

2. TEXT CHARACTERISTICS
   • Average headline length: {df_news['headline_length'].mean():.0f} characters
   • Average words per headline: {df_news['word_count'].mean():.1f}
   • Most common keywords: {', '.join([w[0] for w in top_keywords[:5]])}

3. PUBLISHER INSIGHTS
   • Most active publisher: {publisher_counts.index[0]} ({publisher_counts.values[0]} articles)
   • Top 3 publishers: {', '.join(publisher_counts.head(3).index.tolist())}
   • Publisher concentration: Top 3 publishers = {publisher_counts.head(3).sum() / len(df_news) * 100:.1f}%

4. STOCK COVERAGE
   • Most covered stock: {stock_counts.index[0]} ({stock_counts.values[0]} articles)
   • Top 3 stocks: {', '.join(stock_counts.head(3).index.tolist())}
   • Coverage balance: Gini coefficient analysis shows {('balanced' if stock_counts.std() / stock_counts.mean() < 0.8 else 'imbalanced')} coverage

5. TEMPORAL PATTERNS
   • Average articles per day: {daily_counts.mean():.1f}
   • Daily variation: {daily_counts.std():.1f} (std dev)
   • Peak activity: {daily_counts.max()} articles on {daily_counts.idxmax()}
   • Trend: {'Increasing' if daily_counts.iloc[-30:].mean() > daily_counts.iloc[:30].mean() else 'Decreasing'} over recent 30 days

6. KEY INSIGHTS FOR NEXT STEPS
   • Data quality is good with minimal missing values
   • {df_news['publisher'].nunique()} publishers provide diverse coverage
   • Time series shows {('seasonal' if daily_counts.std() > 0.3 * daily_counts.mean() else 'stable')} patterns
   • Ready for sentiment analysis and correlation study
"""

print(summary)

# Save summary statistics
summary_stats = {
    'Total Articles': len(df_news),
    'Unique Stocks': df_news['stock'].nunique(),
    'Unique Publishers': df_news['publisher'].nunique(),
    'Avg Headline Length': df_news['headline_length'].mean(),
    'Avg Words per Headline': df_news['word_count'].mean(),
    'Articles per Day': daily_counts.mean(),
}

print("\n" + "=" * 70)
print("NEXT STEPS")
print("=" * 70)
print("""
1. ✓ Task 1 - EDA: COMPLETED
   • Loaded and explored financial news dataset
   • Analyzed text characteristics and publisher patterns
   • Identified temporal trends and keyword themes
   • All data quality checks passed

2. → Task 2 - Technical Indicators: NEXT
   • Load historical stock price data via yfinance
   • Calculate TA-Lib indicators (MA, RSI, MACD)
   • Visualize price movements and indicators
   • Prepare data for correlation analysis

3. → Task 3 - Sentiment & Correlation: FINAL
   • Perform sentiment analysis on headlines
   • Align news dates with trading dates
   • Calculate correlation between sentiment and returns
   • Develop trading strategy recommendations
""")

## 9. Summary of Findings

Compile key insights from the exploratory analysis.

## 8. Cross-Sectional Analysis

Analyze relationships between multiple variables.

## 7. Keyword and Topic Analysis

Extract and analyze common keywords and phrases in headlines.

## 6. Temporal Analysis

Analyze publication patterns over time and identify spikes in news activity.

## 5. Stock Coverage Analysis

Analyze which stocks get the most media coverage.

## 4. Publisher Analysis

Identify the most active publishers and their contribution patterns.

## 3. Descriptive Statistics - Text Analysis

Analyze headline characteristics and basic text statistics.

## 2. Data Quality Assessment

Check for missing values, data types, and overall data integrity.

### Load Sample Data

For this demonstration, we'll create and load sample financial news data. In production, replace this with your actual FNSPID dataset.

## 1. Environment Setup and Data Loading

First, we'll install required libraries and configure the environment for analysis.